<img src="../images/pokeball.png" width="100" align="right">

# Visualization of the<br> Favorite Pok&eacute;mon Survey Results
---
In this notebook, I will generate different visualizations of the [Favorite Pok&eacute;mon Survey](<https://www.reddit.com/r/pokemon/comments/c04rvq/survey_testing_the_every_pok%C3%A9mon_is_someones/>) results. The data was collected and made public by [reddit user mamamia1001](https://www.reddit.com/user/mamamia1001/). Full credit goes to him/her. I am just making those numbers look a bit prettier. For more info, take a look at the [README file](../README.md). Let's get started.


# Preliminaries
First, let's import all the relevant packages and define some basic (path) variables.

In [2]:
import pandas as pd
import pathlib

# Define paths.
PATH_DATA = pathlib.Path(r'../data')

# Read data
I [downloaded a copy](../data) of the [original results](https://docs.google.com/spreadsheets/d/1c16Wh4AawHGbTi3Eq1DGZQdM4FMUlJO1YwXJZ_ylRvg/edit#gid=557303698) as an Excel file. Let's read just the relevant part:

In [11]:
df = pd.read_excel(PATH_DATA/'responses.xlsx', sheet_name='Results', usecols='A:B')
df.rename(columns={'Results in full':'name', 'Unnamed: 1':'votes'}, inplace=True)
df.head(5)

,name,votes
0,Bulbasaur,710.0
1,Ivysaur,83.0
2,Venusaur,127.0
3,Charmander,374.0
4,Charmeleon,70.0


In [3]:
%qtconsole